In [10]:
import json
from pathlib import Path
import pandas as pd

print(pd.__version__)

1.3.5


In [57]:
dir_meta = Path('meta')
dir_meta.mkdir(exist_ok=True)

### Data Source
- [iNaturalist 2021 Dataset](https://github.com/visipedia/inat_comp/tree/master/2021)  

- There is a total of 10,000 species in the dataset. The full training dataset contains nearly 2.7M images.  
- To make the dataset more accessible we have also created a "mini" training dataset with 50 examples per species for a total of 500K images.
    - Each species has 10 validation images. There are a total of 500,000 test images.

### Data Storage
- server : A40
- AI_projects/shared/iNaturalist_2021/
- `wget ${file link}`

# Untar Aves imags

### check data structure in compressed Directory
- Untaring the images creates a directory structure like train/category/image.jpg.
- Untaring imgs by (birds) directory
    - filter directory name include "aves", then untar it by keywaord



- Linux tar cmd: 
    - Check how many files contains "Aves"
        - `tar -tvf file.tar.gz |grep Aves`
    - Check how many directory contains "Aves"
        - `tar -tvf file.tar.gz |grep Aves | grep -vc jpg`

    - untar specific files
        - `tar -zxvf file.tar.gz --wi
        ldcards "*Aves*"`
    - [UNIX/Linux 檔案壓縮與備份工具 tar 指令使用教學與範例（一）：tar 檔案](https://blog.gtwang.org/linux/tar-command-examples-in-linux-1/)


# Convert the json files to easy-to-use csv files

### check data structure in jason files

In [22]:
cwd_dir = Path('.')
cwd_files = [ path for path in cwd_dir.glob('*.json')]
print(cwd_files) 

[WindowsPath('public_test.json'),
 WindowsPath('train.json'),
 WindowsPath('train_mini.json'),
 WindowsPath('val.json')]

- keys in json files:
    - 'info'
    - 'images'
    - 'categories'
    - 'annotations'
    - 'licenses'

In [38]:
for file in cwd_files:
    print(file)
    with open(file) as json_data:
        data = json.load(json_data)
        for keys in data.keys():
            print(f'\t{keys:12s}: {len(data[keys]):>8,d}')


public_test.json
	info        :        6
	images      :  500,000
	categories  :   10,000
	licenses    :        7
train.json
	info        :        6
	images      : 2,686,843
	categories  :   10,000
	annotations : 2,686,843
	licenses    :        7
train_mini.json
	info        :        6
	images      :  500,000
	categories  :   10,000
	annotations :  500,000
	licenses    :        7
val.json
	info        :        6
	images      :  100,000
	categories  :   10,000
	annotations :  100,000
	licenses    :        7


#### data structure in jason files
- 'info',  'licenses', is removable
- 'categories' is repeated/sharable
- 'annotations' 
    - Provides information about 'image_id' linked to 'category_id' (which contain taxon information)
    - 'annotations' is lack of in 'public_test.json'
        - public test image don't have taxon information
    


### Establish a shared 'categories' dataframe/csv file
- all sps
- only avian

In [53]:
# with open('val.json') as json_data:
#     data = json.load(json_data)['categories']
#     df_categories = pd.DataFrame.from_dict(data)
    

In [120]:
# df_categories.to_csv(dir_meta/'categories.csv'),
cols = ['supercategory', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus']
dtype = dtype = {col : dtype for col, dtype in zip(cols, ['category']*len(cols))}
df_categories = pd.read_csv(dir_meta/'categories.csv', dtype=dtype)
df_categories

,id,name,common_name,supercategory,kingdom,phylum,class,order,family,genus,specific_epithet,image_dir_name
0,0,Lumbricus terrestris,Common Earthworm,Animalia,Animalia,Annelida,Clitellata,Haplotaxida,Lumbricidae,Lumbricus,terrestris,00000_Animalia_Annelida_Clitellata_Haplotaxida...
1,1,Sabella spallanzanii,Mediterranean Fanworm,Animalia,Animalia,Annelida,Polychaeta,Sabellida,Sabellidae,Sabella,spallanzanii,00001_Animalia_Annelida_Polychaeta_Sabellida_S...
2,2,Serpula columbiana,Serpula columbiana,Animalia,Animalia,Annelida,Polychaeta,Sabellida,Serpulidae,Serpula,columbiana,00002_Animalia_Annelida_Polychaeta_Sabellida_S...
3,3,Spirobranchus cariniferus,Blue Tube Worm,Animalia,Animalia,Annelida,Polychaeta,Sabellida,Serpulidae,Spirobranchus,cariniferus,00003_Animalia_Annelida_Polychaeta_Sabellida_S...
4,4,Eratigena duellica,Giant House Spider,Arachnids,Animalia,Arthropoda,Arachnida,Araneae,Agelenidae,Eratigena,duellica,00004_Animalia_Arthropoda_Arachnida_Araneae_Ag...
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,Psilotum nudum,skeleton fork fern,Plants,Plantae,Tracheophyta,Polypodiopsida,Psilotales,Psilotaceae,Psilotum,nudum,09995_Plantae_Tracheophyta_Polypodiopsida_Psil...
9996,9996,Tmesipteris elongata,Tmesipteris elongata,Plants,Plantae,Tracheophyta,Polypodiopsida,Psilotales,Psilotaceae,Tmesipteris,elongata,09996_Plantae_Tracheophyta_Polypodiopsida_Psil...
9997,9997,Azolla filiculoides,water fern,Plants,Plantae,Tracheophyta,Polypodiopsida,Salviniales,Salviniaceae,Azolla,filiculoides,09997_Plantae_Tracheophyta_Polypodiopsida_Salv...
9998,9998,Salvinia minima,water spangles,Plants,Plantae,Tracheophyta,Polypodiopsida,Salviniales,Salviniaceae,Salvinia,minima,09998_Plantae_Tracheophyta_Polypodiopsida_Salv...


In [121]:
df_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id                10000 non-null  int64   
 1   name              10000 non-null  object  
 2   common_name       10000 non-null  object  
 3   supercategory     10000 non-null  category
 4   kingdom           10000 non-null  category
 5   phylum            10000 non-null  category
 6   class             10000 non-null  category
 7   order             10000 non-null  category
 8   family            10000 non-null  category
 9   genus             10000 non-null  category
 10  specific_epithet  10000 non-null  object  
 11  image_dir_name    10000 non-null  object  
dtypes: category(7), int64(1), object(4)
memory usage: 710.2+ KB


In [122]:
df_categories.describe()

,id
count,10000.00000
mean,4999.50000
std,2886.89568
min,0.00000
25%,2499.75000
50%,4999.50000
75%,7499.25000
max,9999.00000


In [65]:
df_categories.iloc[:,1:].describe()

,name,common_name,supercategory,kingdom,phylum,class,order,family,genus,specific_epithet,image_dir_name
count,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
unique,10000,9962,11,3,13,51,273,1103,4884,6485,10000
top,Lumbricus terrestris,Green Tree Frog,Plants,Animalia,Tracheophyta,Magnoliopsida,Lepidoptera,Asteraceae,Quercus,californica,00000_Animalia_Annelida_Clitellata_Haplotaxida...
freq,1,2,4271,5388,4218,3295,1430,495,43,52,1


In [88]:
print('supercategory : ', df_categories.supercategory.unique())
print('class : ',df_categories['class'].unique())

assert len(df_categories.query('supercategory == "Birds"')) == len(df_categories[df_categories["class"] == "Aves"])

print('Number of bird sps : ', len(df_categories.query('supercategory == "Birds"')))

supercategory :  ['Animalia' 'Arachnids' 'Insects' 'Ray-finned Fishes' 'Amphibians' 'Birds'
 'Mammals' 'Reptiles' 'Mollusks' 'Fungi' 'Plants']
class :  ['Clitellata' 'Polychaeta' 'Arachnida' 'Chilopoda' 'Diplopoda'
 'Hexanauplia' 'Insecta' 'Malacostraca' 'Merostomata' 'Actinopterygii'
 'Amphibia' 'Ascidiacea' 'Aves' 'Elasmobranchii' 'Mammalia' 'Reptilia'
 'Anthozoa' 'Hydrozoa' 'Scyphozoa' 'Asteroidea' 'Echinoidea'
 'Holothuroidea' 'Ophiuroidea' 'Bivalvia' 'Cephalopoda' 'Gastropoda'
 'Polyplacophora' 'Arthoniomycetes' 'Dothideomycetes' 'Lecanoromycetes'
 'Leotiomycetes' 'Pezizomycetes' 'Sordariomycetes' 'Agaricomycetes'
 'Dacrymycetes' 'Pucciniomycetes' 'Tremellomycetes' 'Bryopsida'
 'Polytrichopsida' 'Sphagnopsida' 'Ulvophyceae' 'Jungermanniopsida'
 'Marchantiopsida' 'Florideophyceae' 'Cycadopsida' 'Gnetopsida'
 'Liliopsida' 'Lycopodiopsida' 'Magnoliopsida' 'Pinopsida'
 'Polypodiopsida']
Number of bird sps :  1486


# Establish the Avian metadata
- join 'images' and 'categories' by 'annotations'
- include: 'train.json', 'train_mini.json', 'val.json'

In [151]:
df_birds_cat = (df_categories.query('supercategory == "Birds"')
                .rename({'id':'category_id'}, axis='columns')
                )
df_birds_cat

,category_id,name,common_name,supercategory,kingdom,phylum,class,order,family,genus,specific_epithet,image_dir_name
3111,3111,Accipiter badius,Shikra,Birds,Animalia,Chordata,Aves,Accipitriformes,Accipitridae,Accipiter,badius,03111_Animalia_Chordata_Aves_Accipitriformes_A...
3112,3112,Accipiter cooperii,Cooper's Hawk,Birds,Animalia,Chordata,Aves,Accipitriformes,Accipitridae,Accipiter,cooperii,03112_Animalia_Chordata_Aves_Accipitriformes_A...
3113,3113,Accipiter gentilis,Northern Goshawk,Birds,Animalia,Chordata,Aves,Accipitriformes,Accipitridae,Accipiter,gentilis,03113_Animalia_Chordata_Aves_Accipitriformes_A...
3114,3114,Accipiter nisus,Eurasian Sparrowhawk,Birds,Animalia,Chordata,Aves,Accipitriformes,Accipitridae,Accipiter,nisus,03114_Animalia_Chordata_Aves_Accipitriformes_A...
3115,3115,Accipiter striatus,Sharp-shinned Hawk,Birds,Animalia,Chordata,Aves,Accipitriformes,Accipitridae,Accipiter,striatus,03115_Animalia_Chordata_Aves_Accipitriformes_A...
...,...,...,...,...,...,...,...,...,...,...,...,...
4592,4592,Trogon collaris,Collared Trogon,Birds,Animalia,Chordata,Aves,Trogoniformes,Trogonidae,Trogon,collaris,04592_Animalia_Chordata_Aves_Trogoniformes_Tro...
4593,4593,Trogon elegans,Elegant Trogon,Birds,Animalia,Chordata,Aves,Trogoniformes,Trogonidae,Trogon,elegans,04593_Animalia_Chordata_Aves_Trogoniformes_Tro...
4594,4594,Trogon massena,Slaty-tailed Trogon,Birds,Animalia,Chordata,Aves,Trogoniformes,Trogonidae,Trogon,massena,04594_Animalia_Chordata_Aves_Trogoniformes_Tro...
4595,4595,Trogon melanocephalus,Black-headed Trogon,Birds,Animalia,Chordata,Aves,Trogoniformes,Trogonidae,Trogon,melanocephalus,04595_Animalia_Chordata_Aves_Trogoniformes_Tro...


In [160]:
def get_meta(file_path:str) -> pd.DataFrame:
    with open(file_path) as json_data:
        data = json.load(json_data)
        # df_imgs = pd.DataFrame.from_dict(data['images']) 
        # df_annotations = pd.DataFrame.from_dict(data['annotations'])
        df = (pd.merge(pd.DataFrame.from_dict(data['annotations'])[['id', 'category_id']],
                      pd.DataFrame.from_dict(data['images']),
                      on='id')
              .merge(df_birds_cat, on='category_id', how='inner')
              )
    
    return df

In [155]:
# json_files = ['val.json', 'train_mini.json', 'train.json']

# df_meta_all = pd.concat(
#     [get_meta(file) for file in json_files]
#     )


In [161]:
json_files = ['val.json', 'train_mini.json', 'train.json']
for file in json_files:
    print(file)
    df_meta = get_meta(file)
    file_name = file.split('.')[0]
    df_meta.to_csv(f'meta/{file_name}.csv')




val.json
train_mini.json
train.json


In [167]:
df_meta_train = pd.read_csv(f'meta/train.csv', index_col=0)

In [168]:
df_meta_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414847 entries, 0 to 414846
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    414847 non-null  int64  
 1   category_id           414847 non-null  int64  
 2   width                 414847 non-null  int64  
 3   height                414847 non-null  int64  
 4   file_name             414847 non-null  object 
 5   license               414847 non-null  int64  
 6   rights_holder         414847 non-null  object 
 7   date                  414847 non-null  object 
 8   latitude              413959 non-null  float64
 9   longitude             413959 non-null  float64
 10  location_uncertainty  348357 non-null  float64
 11  name                  414847 non-null  object 
 12  common_name           414847 non-null  object 
 13  supercategory         414847 non-null  object 
 14  kingdom               414847 non-null  object 
 15  